<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/Project/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap
!pip install scikeras

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from xgboost import plot_importance
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils.vis_utils import plot_model

import statistics
from matplotlib import pyplot
import seaborn as sns
import shap
import warnings
import re
warnings.filterwarnings('ignore')

In [1]:
new_df3 = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/data_modelo.csv",sep=";")
base_total_max = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/equipos.csv",sep=";")

NameError: ignored

In [ ]:
X_base_model = new_df3.drop(columns=['team', 'team2', 'results', 'neutral_location'])
Y_model = new_df3['results']
Y_model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_base_model, Y_model, test_size=0.2, random_state=2022)

In [ ]:
y_dict = [{'':str(x)} for x in y_train.tolist()]
y_dict_test = [{'':str(x)} for x in y_test.tolist()]
vec = DictVectorizer(sparse=False, dtype=int)
M_train = vec.fit_transform(y_dict)
M_test = vec.fit_transform(y_dict_test)

In [ ]:
def get_model(meta, activation, n_layers, hidden_layer_1, hidden_layer_2 = None, hidden_layer_3 = None):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = [hidden_layer_1, hidden_layer_2, hidden_layer_3]

    model = Sequential(name="Red_CV")
    model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
    
    model.add(Dense(3, activation='softmax'))
    return model

In [ ]:
clf = KerasClassifier(
    get_model,
    loss="categorical_crossentropy",
    n_layers=2,
    hidden_layer_1=30, 
    hidden_layer_2=10,
    hidden_layer_3=None,
    metrics=["accuracy"],
    epochs=10,
    activation="relu"
)

clf

In [ ]:
ct = ColumnTransformer([
       ('encoder', OneHotEncoder(handle_unknown='ignore'),
       make_column_selector(dtype_include=np.object)),
       ], remainder='passthrough')
estimators = [
              ('encoder', ct),
              ('clf', clf),
              ]

pipe = Pipeline(estimators)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_train)
y_pred

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
with tf.device('/device:GPU:0'): 
    params = {
              "clf__hidden_layer_1": [8, 100, 10, 5],
              "clf__hidden_layer_2": [8, 100, 10, 5],
              "clf__hidden_layer_3": [8, 100, 10, 5],
              "clf__n_layers": [1,2,3],
              "clf__activation": ["relu", "linear", "sigmoid"],
              "clf__optimizer": ["adam", "sgd"],
              "clf__optimizer__learning_rate": [0.0001, 0.001, 0.1],
              "clf__epochs": [10, 30, 20],
             }
    grid = RandomizedSearchCV(pipe, params, cv=3, scoring='neg_log_loss', n_iter=10, random_state=2022)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
best_model = grid.best_estimator_

grid.best_params_

In [ ]:
y_pred = best_model.predict(X_train)
y_pred
print(classification_report(y_train, y_pred))

In [ ]:
y_pred = best_model.predict(X_test)
y_pred
print(classification_report(y_test, y_pred))

In [ ]:
def obtain_new(equipo1, equipo2):
    base1 = base_total_max[base_total_max['team'] == equipo1]
    base2 = base_total_max[base_total_max['team'] == equipo2]
    return base1